In [1]:
!pip install "notebook>=5.3" "ipywidgets>=7.5"
!pip install plotly==5.3.1 

In [2]:
import json
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import plotly.express as px

In [3]:
px.set_mapbox_access_token("pk.eyJ1IjoicmV4dGVyZHMiLCJhIjoiY2t2dWgwazZ2M3E2YTJucWd3aDY1Yzd3dyJ9.r-rgjBmvcCMR1tBu8imGUQ") 

In [4]:
sc = SparkContext()
ss = SparkSession(sc)


In [5]:
business = ss.read.json("data/yelp_academic_dataset_business.json")
checkin = ss.read.json("data/yelp_academic_dataset_checkin.json")
review = ss.read.json("data/yelp_academic_dataset_review.json")
tip = ss.read.json("data/yelp_academic_dataset_tip.json")
user = ss.read.json("data/yelp_academic_dataset_user.json")

In [6]:
business.printSchema()
print(business.count())

root
 |-- _corrupt_record: string (nullable = true)
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string 

In [7]:
checkin.printSchema()
print(checkin.count())

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)

138876


In [8]:
review.printSchema()
print(review.count())

root
 |-- _corrupt_record: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

8618983


In [9]:
tip.printSchema()
print(tip.count())

root
 |-- _corrupt_record: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- compliment_count: long (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)

1160282


In [10]:
user.printSchema()
print(user.count())

root
 |-- _corrupt_record: string (nullable = true)
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)

2188437


In [11]:
review_counts = (business
                  .select("name", "latitude", "longitude", "review_count", "stars", "state")
                  .sort("state", "review_count", ascending=False))
review_counts.show()

+--------------------+-------------+---------------+------------+-----+-----+
|                name|     latitude|      longitude|review_count|stars|state|
+--------------------+-------------+---------------+------------+-----+-----+
|Good Times Burger...|    40.022457|    -105.260884|           9|  1.5|   WY|
|     Brooks Brothers|   42.3586616|    -71.0550652|           7|  3.5|   WI|
|Beaches Restauran...|   45.6148087|   -122.6520268|         719|  3.5|   WA|
|Kenji's Ramen & G...|   45.6197591|   -122.5399554|         657|  4.0|   WA|
|Thai Orchid Resta...|   45.6295205|   -122.6728246|         636|  4.0|   WA|
|    Thai Wok Kitchen|45.6412270338|-122.6175698488|         556|  4.5|   WA|
|    Joe's Crab Shack|45.6209771132|   -122.6715666|         550|  3.0|   WA|
|Abhiruchi Indian ...|   45.6229319|   -122.5565457|         497|  4.5|   WA|
|Lapellah Restaura...|   45.6191266|   -122.6442021|         465|  3.5|   WA|
|The Rock Wood Fir...|45.6191484137|-122.6456597789|         465

In [12]:
similar_categories = (business
                     .select("name", "categories", "state", "stars")
                     .sort("categories", "state", ascending=False))
similar_categories.show()

+--------------------+--------------------+-----+-----+
|                name|          categories|state|stars|
+--------------------+--------------------+-----+-----+
|Nutria Pond behin...|Zoos, Parks, Acti...|   OR|  3.5|
|          Oregon Zoo|Zoos, Parks, Acti...|   OR|  4.0|
|Manatee Observati...|Zoos, Local Servi...|   FL|  4.5|
|Rose and Grace Mi...|Zoos, Event Plann...|   TX|  5.0|
|   Jungle Adventures|Zoos, Active Life...|   FL|  4.0|
|   Franklin Park Zoo|   Zoos, Active Life|   MA|  3.5|
|Maharajah Jungle ...|   Zoos, Active Life|   FL|  4.5|
|Zoom Air Adventur...|Ziplining, Botani...|   FL|  4.5|
|   Orlando Tree Trek|Ziplining, Active...|   FL|  4.5|
|Grouse Mountain Z...|Ziplining, Active...|   BC|  4.0|
|          LA Fitness|Yoga, Trainers, S...|   OH|  2.0|
|          Cycle Elan|Yoga, Trainers, G...|   GA|  5.0|
|          LA Fitness|Yoga, Trainers, G...|   GA|  2.5|
|   Blueprint Fitness|Yoga, Trainers, G...|   GA|  5.0|
|    Full Circle Yoga|Yoga, Trainers, F...|   FL

In [13]:
business_checkin = (business
                    .join(checkin, "business_id")
                    .withColumn("checkin_count", F.size(F.split(F.col("date"), ",")))
                    .select("business_id", "latitude", "longitude", "checkin_count")
                    .sort("checkin_count", ascending=False))

business_checkin.show()

+--------------------+-------------+---------------+-------------+
|         business_id|     latitude|      longitude|checkin_count|
+--------------------+-------------+---------------+-------------+
|7sxYa0-TwWeWGFr5C...|33.6407227072| -84.4276356697|       150005|
|EqUqaLJxMDxIqvnma...|     45.58979|    -122.595204|        75511|
|2PxZ-fICnd432NJHe...|28.4278388741|  -81.308182904|        63982|
|UlndVD4tezU3FACjZ...|    30.202473|     -97.666861|        54657|
|vITc1UkWLvG9pVdaO...|42.3622379746| -71.0210756391|        54361|
|cyBm7p3D2RAoGlDn-...|    28.417663|     -81.581212|        25208|
|NvAYQvjLxwGC-kcWz...|    28.374694|     -81.549404|        24017|
|4CxF8c3MB7VAdY8zF...|45.5226120114|-122.6730836114|        20291|
|OPfgKOm_n-ajUo3qj...|   45.5230969|    -122.681325|        19941|
|m3DeBd0NSbAGDjUOk...|49.1942850587|  -123.18076195|        18388|
|BOFD5UmhHvCn_XH3_...|    28.370971|     -81.519392|        17033|
|C_k727ws82eMe9xtJ...|   39.9980574|    -82.8838088|        16

In [14]:
review_dates = (review
                .join(business, "business_id")
                .select("review_id", "business_id", "date")
                .sort("business_id", "date", ascending=False)
                .withColumn("month", F.split(F.col("date"), "-")[1])
                .withColumn("year", F.split(F.col("date"), "-")[0]))

review_dates.show()

+--------------------+--------------------+-------------------+-----+----+
|           review_id|         business_id|               date|month|year|
+--------------------+--------------------+-------------------+-----+----+
|RIlyZDUUL7dn-wX9R...|zzzKmD9Mj6WtJwJUh...|2012-01-05 07:25:19|   01|2012|
|Beiu8FUpabvKp_tIj...|zzzKmD9Mj6WtJwJUh...|2011-07-06 03:11:19|   07|2011|
|NsbPfBB7VcKavo2kP...|zzzKmD9Mj6WtJwJUh...|2011-01-30 07:26:01|   01|2011|
|ZLg9JZBm8fti3kdcK...|zzzKmD9Mj6WtJwJUh...|2010-12-26 08:06:25|   12|2010|
|_PDg4GKwqstcU4jqS...|zzzKmD9Mj6WtJwJUh...|2010-05-05 21:14:56|   05|2010|
|7Me9k1V0KNyqAds43...|zzzKmD9Mj6WtJwJUh...|2010-03-07 20:52:10|   03|2010|
|PEgXvQE2ZKGQZWqt5...|zzzKmD9Mj6WtJwJUh...|2009-10-22 19:53:49|   10|2009|
|coFCcXcoVnTLwNMct...|zzzKmD9Mj6WtJwJUh...|2009-10-13 08:08:57|   10|2009|
|79BNQZEWEcZjZe4G7...|zzxIO4bjWCQrvCJ0O...|2020-06-05 12:17:10|   06|2020|
|DWbIic86a-H_8nau6...|zzxIO4bjWCQrvCJ0O...|2020-01-09 21:53:36|   01|2020|
|6LVhP-V7QtsjN34Ao...|zzx

In [15]:
review_count_by_date = (review_dates
                        .select("month", "year")
                        .sort("year", "month", ascending=False))

review_count_by_date.show()

review_count_by_date.groupBy("month").pivot("year").count().sort("month", ascending=True).show()

+-----+----+
|month|year|
+-----+----+
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
|   01|2021|
+-----+----+
only showing top 20 rows

+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+
|month|2004|2005|2006|2007| 2008| 2009| 2010| 2011| 2012| 2013| 2014| 2015| 2016| 2017|  2018| 2019| 2020| 2021|
+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+
|   01|null|   1| 829|4641| 9978|17157|23355|33857|39638|42999|52222|68179|79224|77022| 82318|83009|72710|42385|
|   02|null|   4|1242|4374| 9627|15091|22542|32067|36790|39963|47830|61077|75043|72542| 77057|77788|70992| null|
|   03|null|  25|1466|4431|10509|16563|25287|36408|40011|43022|56319|73258|78672|84138| 90887|893

In [16]:
user_reviews_location = (user
                           .join(review, 'user_id')
                           .join(business, 'business_id'))

user_reviews_location.select('user_id', 'business_id', 'latitude', 'longitude').show()

+--------------------+--------------------+-------------+---------------+
|             user_id|         business_id|     latitude|      longitude|
+--------------------+--------------------+-------------+---------------+
|--0666UC6t7DJWO4m...|5xS76cdT7THG14H_W...|    28.359719|     -81.591313|
|--0LWpVOR539pgBvl...|QVnatPo7jCEh9FcKl...|   42.3647559|    -71.1032591|
|--19CSe_41_fEKvfV...|i7jMAf1cZJ9h1FRTQ...|40.0176800756|-105.2803100092|
|--2GHi3yi8D4yBuUY...|lAMX7ndl9sJh117Qh...|   45.3838053|   -122.7590583|
|--2GHi3yi8D4yBuUY...|1jW2Voh5XE7Yt6ZV-...|45.5735589587|-122.5590787902|
|--56dSK1Y7agEMyIJ...|ZHxrgZY4NAqtnsBuo...|30.3322467627| -97.7405043632|
|--9BvAldSQW8wmAn0...|ebCy57mw7WPLrnWLQ...|  45.36711897|  -122.60042912|
|--DBsOOiMurIanfF2...|-2aMFgYft7rED0bst...|    28.358601|     -81.333251|
|--HBdJ3-gCfIwC3aA...|lslTGfavHimYBM8fT...|     45.51885|   -122.6788921|
|--Hh_cXFJJUqYB2ST...|Tav-9OWqGKFestUqt...|   45.5197485|   -122.6773629|
|--IpFJ0EzvdepaxP4...|2kKbqNPY6j1AbpXx

In [ ]:
review_counts_pd = review_counts.toPandas()
fig = px.density_mapbox(review_counts_pd, lat='latitude', lon='longitude', z='review_count', radius=10, zoom=3,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
business_checkin_pd = business_checkin.toPandas()

fig = px.density_mapbox(business_checkin_pd, lat='latitude', lon='longitude', z='checkin_count', radius=10, zoom=3,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
fig = px.scatter_mapbox(business_checkin_pd, lat='latitude', lon='longitude', color="checkin_count", size="checkin_count",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()